In [ ]:
import pandas as pd
import sqlite3
import sqlalchemy

In [ ]:
class Split(object):
    def __init__(self, *args, **kwargs):
        id: int = None
        journal_entry_key: int = None
        account_key: int = None
        # account_name: str = None
        date = None
        description: str = ''
        memo: str = 
        debit: float = 0.0
        credit: float = 0.0
        

In [ ]:
import pandas as pd

class JournalEntry(object):
    columns = id', 'date', 'date_entered', 'date_modified', 'description', 'memo', 'debit', 'credit']
    dtypes = 'int64', 'datetime64 ns]', 'datetime64[ns]', 'datetime64[ns]', 'object', 'object', 'float64', 'float64']
    df = pd.DataFrame(columns=columns).astype(dict(zip(columns, dtypes)))


        

In [ ]:
import pandas as pd
JournalEntry_columns=['id', 'date', 'date_entered', 'date_modified', 'description', 'memo', 'debit', 'credit'
JournalEntry = pd.DataFrame(columns=JournalEntry_columns)  # , dtype=['int64', 'datetime64[ns', 'datetime64[ns]', 'datetime64[ns]', 'object', 'object', 'float64', 'float64'])
JournalEntry_dtypes = ['int64', 'datetime64[ns]', 'datetime64[ns]', 'datetime64[ns]', 'object', 'object', 'float64', 'float64']

JournalEntry = JournalEntry.astype(dict(zip(JournalEntry_columns, JournalEntry_dtypes)))

JournalEntry.dtypes

id                        int64
date             datetime64[ns]
date_entered     datetime64[ns]
date_modified    datetime64[ns]
description              object
memo                     object
debit                   float64
credit                  float64
dtype: object

In [27]:
#df.columns = JournalEntry.columns
journal_entries = pd.DataFrame()
d = dict(zip(JournalEntry.columns, JournalEntry.dtypes))
journal_entries = journal_entries.assign(**{col: pd.Series(dtype=d[col]) for col in JournalEntry.columns})
journal_entries

,id,date,date_entered,date_modified,description,memo,debit,credit


In [ ]:
class AcctTable(object):
    columns = None
    dtypes = None
    map = None
    @classmethod
    def create(cls):
        df = pd.DataFrame()
        df = df.assign(**{col: pd.Series(dtype=cls.map[col]) for col in cls.columns})
        return df

class Company(AcctTable):
    columns = ['id', 'coa_key', 'account_key', 'company_name', 'company_type', 'date', 'description']
    dtypes = ['int64', 'int64', 'int64', 'object', 'object', 'datetime64[ns]', 'object']
    map = dict(zip(columns, dtypes))


class ChartOfAccounts(AcctTable):
    columns = ['id', 'company_key']
    dtypes = ['int64', 'int64']
    map = dict(zip(columns, dtypes))


class Accounts(AcctTable):
    columns = ['id', 'company_key', 'coa_key', 'parent_account_key', 'name', 'number', 'type', 'is_container', 'is_root', 'date_created', 'date_modified', 'description']
    dtypes = ['int64', 'int64', 'int64', 'int64', 'object', 'int64', 'object', 'boolean', 'boolean', 'datetime64[ns]', 'datetime64[ns]', 'object']
    map = dict(zip(columns, dtypes))


class JournalEntries(AcctTable):
    columns = ['id', 'date', 'date_entered', 'date_modified', 'description', 'memo', 'debit', 'credit']
    dtypes = ['int64', 'datetime64[ns]', 'datetime64[ns]', 'datetime64[ns]', 'object', 'object', 'float64', 'float64']
    map = dict(zip(columns, dtypes))


class Splits(AcctTable):
    columns = ['id', 'journal_entry_key', 'account_key', 'account_name', 'date', 'description', 'memo', 'debit', 'credit']
    dtypes = ['int64', 'int64', 'int64', 'object', 'datetime64[ns]', 'object', 'object', 'float64', 'float64']
    map = dict(zip(columns, dtypes))



class Books(object):
    company: pd.DataFrame = None
    chart_of_accounts: pd.DataFrame = None
    accounts: pd.DataFrame= None
    journal_entries: pd.DataFrame = None
    splits: pd.DataFrame = None
    tables: dict = dict(company=company, chart_of_accounts=chart_of_accounts, accounts=accounts, journal_entries=journal_entries, splits=splits)
    classes: dict = dict(company=Company, chart_of_accounts=ChartOfAccounts, accounts=Accounts, journal_entries=JournalEntries, splits=Splits)
    loadtime = None

    @classmethod
    def load(cls, file='savefile.account'):
        import sqlalchemy, os, datetime

        if not os.path.exists(file):
            for name in cls.tables.keys():
                ## Create empty from scratch
                if cls.classes[name] is None or isinstance(cls.classes[name], str):
                    continue
                df = pd.DataFrame()
                df = df.assign(**{col: pd.Series(dtype=cls.classes[name].map[col]) for col in cls.classes[name].columns})
                setattr(cls, name, df)
                cls.tables[name] = df
            cls.loadtime = datetime.datetime.now()
            return  ## must return here

        ## Path exists.  Load from file.
        engine = sqlalchemy.create_engine(f'sqlite:///{file}', echo=False)

        for name, data in cls.tables.items():
            table = pd.read_sql(sql=f'SELECT * FROM {name}', con=engine)
            table = table.astype(cls.classes[name].map)
            setattr(cls, name, table)
            cls.tables[name] = table
        
        cls.loadtime = datetime.datetime.now()


    @classmethod
    def save(cls, file='savefile.account'):
        ## Move previous save and include a timestamp in its filename
        import shutil, sqlalchemy, os

        if os.path.exists(file):
            datetime_format = '%Y-%m-%dT%H_%M_%S.%f'
            backup_filename = f'{file[:file.rfind('.')]}.{cls.loadtime.strftime(datetime_format)}{file[file.rfind('.'):]}'
            shutil.move(src=file, dst=backup_filename)
        
        ## Save existing tables
        engine = sqlalchemy.create_engine(f'sqlite:///{file}', echo=False)
        for name, data in cls.tables.items():
            data.to_sql(con=engine, name=name)

    



In [115]:
Books.load()
Books.tables
Books.save()

In [114]:
Books.tables

{'company': Empty DataFrame
 Columns: [level_0, index, id, coa_key, account_key, company_name, company_type, date, description]
 Index: [],
 'chart_of_accounts': Empty DataFrame
 Columns: [level_0, index, id, company_key]
 Index: [],
 'accounts': Empty DataFrame
 Columns: [level_0, index, id, company_key, coa_key, parent_account_key, name, number, type, is_container, is_root, date_created, date_modified, description]
 Index: [],
 'journal_entries': Empty DataFrame
 Columns: [level_0, index, id, date, date_entered, date_modified, description, memo, debit, credit]
 Index: [],
 'splits': Empty DataFrame
 Columns: [level_0, index, id, journal_entry_key, account_key, account_name, date, description, memo, debit, credit]
 Index: []}

In [112]:
Books.tables['company'].drop(['level_0'], axis=1)

,index,id,coa_key,account_key,company_name,company_type,date,description


In [104]:
import datetime
Books.tables['company'].iloc[0] = (1, None, None, 'Company1', 'LLC', datetime.datetime.now(), 'Test company')

IndexError: iloc cannot enlarge its target object

In [116]:
from datetime import datetime
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String
from sqlalchemy import Integer
from sqlalchemy import Boolean
from sqlalchemy import Float
from sqlalchemy import func
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    pass

class Company(Base):
    __tablename__ = 'company'
    id: Mapped[int] = mapped_column(primary_key=True)
    coa_key: Mapped[int] = mapped_column(ForeignKey('chart_of_accounts.id'))
    company_name: Mapped[str] = mapped_column(String(250))
    company_type: Mapped[str] = mapped_column(String(50))
    date: Mapped[datetime] = mapped_column(insert_default=func.now())
    description: Mapped[str] = mapped_column(String(1000))

    coa: Mapped["Company"] = relationship(back_populates='company')

    def __repr__(self) -> str:
        return f"Company(id={self.id!r}, company_name={self.company_name!r}, description={self.description!r})"

class ChartOfAccounts(Base):
    __tablename__ = 'chart_of_accounts'
    id: Mapped[int] = mapped_column(primary_key=True)
    company_key: Mapped[int] = mapped_column(ForeignKey('company.id'))

    company: Mapped['Company'] = relationship(back_populates='coa')
    accounts: Mapped[List['Accounts']] = relationship(back_populates='coa')

    def __repr__(self) -> str:
        return f"ChartOfAccounts(id={self.id!r}, description={self.company!r})"


class Accounts(Base):
    __tablename__ = 'accounts'
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    coa_key: Mapped[int] = mapped_column(ForeignKey('chart_of_accounts.id'))
    parent_account_key: Mapped[int] = mapped_column(ForeignKey('accounts.id'))
    name: Mapped[str] = mapped_column(String(50))
    number: Mapped[int] = mapped_column(Integer)
    account_type: Mapped[str] = mapped_column(String(50))
    is_container: Mapped[bool] = mapped_column(Boolean)
    is_root: Mapped[bool] = mapped_column(Boolean)
    date_created: Mapped[datetime] = mapped_column(insert_default=func.now())
    date_modified: Mapped[datetime] = mapped_column(insert_default=func.now())
    description: Mapped[str] = mapped_column(String(250))  

    coa: Mapped['ChartOfAccounts'] = relationship(back_populates='accounts')
    parent_account: Mapped['Accounts'] = relationship(back_populates='parent_account')

    def __repr__(self) -> str:
        return f"Accounts(id={self.id!r}, description={self.description!r}, container={self.is_container!r}, root={self.is_root!r})"


class JournalEntries(Base):
    __tablename__ = 'journal_entries'
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    date: Mapped[datetime] = mapped_column(insert_default=func.now())
    date_entered: Mapped[datetime] = mapped_column(insert_default=func.now())
    date_modified: Mapped[datetime] = mapped_column(insert_default=func.now())
    description: Mapped[str] = mapped_column(String(250))  
    memo: Mapped[str] = mapped_column(String(250))  
    debit: Mapped[float] = mapped_column(Float)
    credit: Mapped[float] = mapped_column(Float)

    splits: Mapped[List['Splits']] = relationship(back_populates='journal_entry')

    def __repr__(self) -> str:
        return f"JournalEntries(id={self.id!r}, description={self.description!r}, debit={self.debit!r}, credit={self.credit!r})"

class Splits(Base):
    __tablename__ = 'splits'
    id: Mapped[int] = mapped_column(Integer, primary_key=True)
    journal_entry_key: Mapped[int] = mapped_column(ForeignKey('journal_entries.id'))
    account_key: Mapped[int] = mapped_column(ForeignKey('accounts.id'))
    date: Mapped[datetime] = mapped_column(insert_default=func.now())
    description: Mapped[str] = mapped_column(String(250)) 
    memo: Mapped[str] = mapped_column(String(250)) 
    debit: Mapped[float] = mapped_column(Float)
    credit: Mapped[float] = mapped_column(Float)

    journal_entry: Mapped['JournalEntries'] = relationship(back_populates='splits')

    def __repr__(self) -> str:
         return f"Splits(id={self.id!r}, description={self.description!r}, debit={self.debit!r}, credit={self.credit!r})"


